**Части работы**

0. Создание файлов
1. Обработка данных
2. Расчёт стоимости
3. Юнит тесты

Итог работы:

Вычесленная стоимость содержания зоопарка = **1609** у.е. в день, юнит тесты подтверждают правильность расчётов и эффективность написанной функции

In [1]:
import numpy as np
import pandas as pd
import csv

# 0. Создание файлов



In [2]:
# Prices

with open('prices.txt', 'w') as file:
    file.write("Meat=12.56\nFruit=5.60")

# Zoo

data = """Animal;Name;Weight
Lion;Leo;160
Lion;Nina;172
Lion;Rex;190
Giraffe;Bella;200
Giraffe;Luna;202
Giraffe;Nina;199
Tiger;Max;150
Tiger;Oscar;142
Tiger;Milo;139
Zebra;Ricky;100
Zebra;Tommy;62
Wolf;Sky;78
Wolf;Rex;69
Piranha;Lily;0.5"""

# Разделение данных по строкам
lines = data.strip().split('\n')

# Создание csv файла и запись в него данных
with open('zoo.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    for line in lines:
        writer.writerow(line.split(';'))

# Animals

data = """Lion;0.10;meat;
Tiger;0.09;meat;
Giraffe;0.08;fruit;
Zebra;0.08;fruit;
Wolf;0.07;both;90%
Piranha;0.5;both;50%"""

# Разделение данных по строкам
lines = data.strip().split('\n')

# Создание csv файла и запись в него данных
with open('animals.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    for line in lines:
        writer.writerow(line.split(';'))

# 1. Обработка данных

In [3]:
# Чтение файла zoo.csv
zoo_data = pd.read_csv('zoo.csv', sep=';')

# Пример вывода содержимого файла zoo.csv
zoo_data


,Animal,Name,Weight
0,Lion,Leo,160.0
1,Lion,Nina,172.0
2,Lion,Rex,190.0
3,Giraffe,Bella,200.0
4,Giraffe,Luna,202.0
5,Giraffe,Nina,199.0
6,Tiger,Max,150.0
7,Tiger,Oscar,142.0
8,Tiger,Milo,139.0
9,Zebra,Ricky,100.0


In [4]:
# Чтение файла animals.csv
animals_data = pd.read_csv('animals.csv', sep=';')

# Пример вывода содержимого файла zoo.csv
animals_data

,Lion,0.10,meat,Unnamed: 3
0,Tiger,0.09,meat,NaN
1,Giraffe,0.08,fruit,NaN
2,Zebra,0.08,fruit,NaN
3,Wolf,0.07,both,90%
4,Piranha,0.50,both,50%


In [5]:
# Заменяем названия колонок
animals_data.columns = ['Animal', 'Weight_coef', 'Food', 'Proportion']

# Добавляем строку
new_row = pd.DataFrame({'Animal': ['Lion'], 'Weight_coef': [0.10], 'Food': ['meat'], 'Proportion': [np.nan]})
animals_data = pd.concat([new_row, animals_data]).reset_index(drop=True)

#Переводим % в десятичную дровь
animals_data['Proportion'] = animals_data['Proportion'].str.replace('%', '').astype(float) / 100


In [6]:
animals_data

,Animal,Weight_coef,Food,Proportion
0,Lion,0.10,meat,NaN
1,Tiger,0.09,meat,NaN
2,Giraffe,0.08,fruit,NaN
3,Zebra,0.08,fruit,NaN
4,Wolf,0.07,both,0.9
5,Piranha,0.50,both,0.5


In [7]:
prices_data = pd.read_csv('prices.txt', sep='=', header=None, names=['Food', 'Price'])


#чтобы название еды совпадало с названием в animals_data
prices_data['Food'] = prices_data['Food'].str.lower()

#добавляем строчку both - иначе таблицы не соединяться

prices_data.loc[len(prices_data)] = 'both', None

In [8]:
prices_data

,Food,Price
0,meat,12.56
1,fruit,5.60
2,both,NaN


# 2. Расчёт стоимости

1. Создадим функцию, которая будет принимать три датафрейма: zoo_data, animals_data, prices_data.
2. Внутри функции, мы сначала объединим данные из всех датафреймов, чтобы получить общую информацию.
3. Затем, для каждого животного, вычислим необходимое количество пищи (Food) в зависимости от его веса и коэффициента.
4. Для всеядных животных, учитывая процент (Proportion), распределим необходимое количество пищи между мясом и фруктами.
5. После этого, вычислим общую стоимость кормления всех животных в зоопарке.

In [9]:
#Как выглядит в целом единая таблица

combined_data = pd.merge(zoo_data, animals_data, on="Animal")
combined_data = pd.merge(combined_data, prices_data, on="Food")
combined_data

#Без добавления both всех всеядных иначе бы выкинуло, поэтому нужно было добавить его в датафрейм prices_data

,Animal,Name,Weight,Weight_coef,Food,Proportion,Price
0,Lion,Leo,160.0,0.10,meat,NaN,12.56
1,Lion,Nina,172.0,0.10,meat,NaN,12.56
2,Lion,Rex,190.0,0.10,meat,NaN,12.56
3,Tiger,Max,150.0,0.09,meat,NaN,12.56
4,Tiger,Oscar,142.0,0.09,meat,NaN,12.56
5,Tiger,Milo,139.0,0.09,meat,NaN,12.56
6,Giraffe,Bella,200.0,0.08,fruit,NaN,5.60
7,Giraffe,Luna,202.0,0.08,fruit,NaN,5.60
8,Giraffe,Nina,199.0,0.08,fruit,NaN,5.60
9,Zebra,Ricky,100.0,0.08,fruit,NaN,5.60


In [10]:
    # Шаг 1: Создаем функцию

def calculate_feeding_cost(zoo_data, animals_data, prices_data):

   # Шаг 2: объединение данных
  combined_data = pd.merge(zoo_data, animals_data, on="Animal")
  combined_data = pd.merge(combined_data, prices_data, on="Food")

  # Шаг 3: вычисление необходимого количества пищи

  combined_data["Required_food"] = combined_data["Weight"] * combined_data["Weight_coef"]

  # Шаг 4: расчет стоимости пищи

  combined_data["Priced_food"] = combined_data.apply(lambda row:
                                                   row["Required_food"] * row["Price"] if pd.isnull(row["Proportion"])
                                                   else row["Required_food"] * (prices_data.loc[prices_data['Food'] == 'fruit', 'Price'].values[0] * (1 - row["Proportion"]) + prices_data.loc[prices_data['Food'] == 'meat', 'Price'].values[0]* row["Proportion"]),
                                                   axis=1) #умножаем пропорцию мяса на цену мяса, пропорцию фруктов на цену фруктов - получаем взвешанную цену еды
  # Шаг 5: вычисление общей стоимости

  total_cost = combined_data["Priced_food"].sum()

  return total_cost



In [11]:
total_feeding_cost = calculate_feeding_cost(zoo_data, animals_data, prices_data)
print("Общая стоимость кормления всех животных в зоопарке:", total_feeding_cost)

Общая стоимость кормления всех животных в зоопарке: 1609.0089600000001


# 3. Юнит тесты

3.1 Первый тест - расчет содержания трех львов

Проверим расчет на "игрушечном" примере

In [12]:
#проверим на простом примере - три льва

zoo_data_test1 = pd.DataFrame({"Animal": ["Lion", "Lion", "Lion"],
                         "Name": ["Leo", "Nina", "Rex"],
                         "Weight": [160.0, 172.0, 190.0]})

animals_data_test1 = pd.DataFrame({"Animal": ["Lion"],
                            "Weight_coef": [0.10],
                            "Food": ["meat"],
                            "Proportion": [None]})

prices_data_test1 = pd.DataFrame({"Food": ["meat"],
                           "Price": [12.56]})

In [13]:
((zoo_data_test1["Weight"]).sum())*animals_data_test1["Weight_coef"]*prices_data_test1["Price"]

0    655.632
dtype: float64

In [14]:
total_feeding_cost = calculate_feeding_cost(zoo_data_test1, animals_data_test1, prices_data_test1)
print("Общая стоимость кормления всех животных в зоопарке:", total_feeding_cost)

Общая стоимость кормления всех животных в зоопарке: 655.6320000000001


Проверка вручную и по заданной функции задали **верный** результат


---




3.2 Второй тест - расчет содержания льва и слона

In [15]:
# Данные для тестирования
zoo_data_test2 = pd.DataFrame({"Animal": ["Lion", "Elephant"], "Weight": [200, 1000]})
animals_data_test2 = pd.DataFrame({"Animal": ["Lion", "Elephant"], "Weight_coef": [0.5, 0.3], "Food": ["meat", "fruit"], "Proportion": [None, None]})
prices_data_test2 = pd.DataFrame({"Food": ["meat", "fruit"], "Price": [10, 5.60]}) #изменим входную цену

expected_result = 200*0.5*10+1000*0.3*5.60

# Выполнение функции
result = calculate_feeding_cost(zoo_data_test2, animals_data_test2, prices_data_test2)

print('Ручной подсчет:', expected_result,'Подсчет функцией:', result)

Ручной подсчет: 2680.0 Подсчет функцией: 2680.0


In [16]:
combined_data_test = pd.merge(zoo_data_test2, animals_data_test2, on="Animal")
combined_data_test = pd.merge(combined_data_test, prices_data_test2, on="Food")
combined_data_test

,Animal,Weight,Weight_coef,Food,Proportion,Price
0,Lion,200,0.5,meat,None,10.0
1,Elephant,1000,0.3,fruit,None,5.6


Проверка вручную и по заданной функции задали **верный** результат


---



3.3 Третий тест: введем новые параметры

In [17]:
zoo_data_test3 = pd.DataFrame({"Animal": ["Lion", "Wolf", 'Elephant'], "Weight": [200, 100, 1000]})
animals_data_test3 = pd.DataFrame({"Animal": ["Lion", "Wolf", "Elephant"],"Name": ['Leo', 'Roger', 'Mikuru'], "Weight_coef": [0.5, 0.2, 0.3], "Food": ["meat", "both", 'fruit'], "Proportion": [None, 0.2, None]})
prices_data_test3 = pd.DataFrame({"Food": ["meat", "fruit", "both"], "Price": [20, 5.60, None]}) #вновь изменим входную цену

expected_result = 200*0.5*20+100*0.2*(5.60*(1-0.2)+20*0.2)+1000*0.3*5.60

# Выполнение функции
result = calculate_feeding_cost(zoo_data_test3, animals_data_test3, prices_data_test3)

print('Ручной подсчет:', expected_result,'Подсчет функцией:', result)

Ручной подсчет: 3849.6 Подсчет функцией: 3849.6


Проверка вручную и по заданной функции задали **верный** результат


---



In [18]:
combined_data_test = pd.merge(zoo_data_test3, animals_data_test3, on="Animal")
combined_data_test = pd.merge(combined_data_test, prices_data_test3, on="Food")
combined_data_test

,Animal,Weight,Name,Weight_coef,Food,Proportion,Price
0,Lion,200,Leo,0.5,meat,NaN,20.0
1,Wolf,100,Roger,0.2,both,0.2,NaN
2,Elephant,1000,Mikuru,0.3,fruit,NaN,5.6


3.4 Четвертый тест: только мясоеды, только плотоядные, только всеядные

Импорт для тестов

In [19]:
import pytest

Общая функция для числового тестирования

In [20]:
def test_feeding_cost(meat_eaters, fruit_eaters, both_eaters, expected_cost):
    zoo_data = pd.DataFrame(meat_eaters + fruit_eaters + both_eaters)
    animals_data = pd.DataFrame({
        'Animal': ['Lion', 'Tiger', 'Giraffe', 'Zebra', 'Wolf', 'Piranha'],
        'Weight_coef': [0.10, 0.09, 0.08, 0.08, 0.07, 0.5],
        'Food': ['meat', 'meat', 'fruit', 'fruit', 'both', 'both'],
        'Proportion': [np.nan, np.nan, np.nan, np.nan, 0.9, 0.5]
    })
    prices_data = pd.DataFrame({
        'Food': ['meat', 'fruit', 'both'],
        'Price': [12.56, 5.60, np.nan]
    })

    total_feeding_cost = calculate_feeding_cost(zoo_data, animals_data, prices_data)
    print('Expected:', expected_cost, 'Actual:', total_feeding_cost)
    assert total_feeding_cost == expected_cost

Проверка подсчёта стоимости только для мясоедов

In [21]:
def test_meat_eaters_only():
    meat_eaters = [{'Animal': 'Lion', 'Name': 'Leo', 'Weight': 160},
                   {'Animal': 'Tiger', 'Name': 'Max', 'Weight': 150}]
    expected_total_cost = 160 * 0.10 * 12.56 + 150 * 0.09 * 12.56
    test_feeding_cost(meat_eaters, [], [], expected_total_cost)

test_meat_eaters_only()

Expected: 370.52 Actual: 370.52


Проверка вручную и по заданной функции задали **верный** результат

---



Проверка подсчёта стоимости только для фруктоедов

In [22]:
def test_fruit_eaters_only():
    fruit_eaters = [{'Animal': 'Giraffe', 'Name': 'Bella', 'Weight': 200},
                    {'Animal': 'Zebra', 'Name': 'Ricky', 'Weight': 100}]
    expected_total_cost = 200 * 0.08 * 5.60 + 100 * 0.08 * 5.60
    test_feeding_cost([], fruit_eaters, [], expected_total_cost)

test_fruit_eaters_only()

Expected: 134.39999999999998 Actual: 134.39999999999998


Проверка вручную и по заданной функции задали **верный** результат


---



Проверка подсчёта стоимости только для всеядных

In [23]:
def test_both_eaters_only():
    both_eaters = [{'Animal': 'Wolf', 'Name': 'Sky', 'Weight': 78},
                   {'Animal': 'Piranha', 'Name': 'Lily', 'Weight': 0.5}]
    expected_total_cost = (
        78 * 0.07 * (12.56 * 0.9 + 5.60 * 0.1) +
        0.5 * 0.5 * (12.56 * 0.5 + 5.60 * 0.5)
    )
    test_feeding_cost([], [], both_eaters, expected_total_cost)

test_both_eaters_only()

Expected: 67.04744000000001 Actual: 67.04744000000001


Проверка вручную и по заданной функции задали **верный** результат


---



Проверка подсчёта стоимости для всех типов животных

In [24]:
def test_all_types_of_eaters():
    meat_eaters = [{'Animal': 'Lion', 'Name': 'Leo', 'Weight': 160},
                   {'Animal': 'Tiger', 'Name': 'Max', 'Weight': 150}]
    fruit_eaters = [{'Animal': 'Giraffe', 'Name': 'Bella', 'Weight': 200},
                    {'Animal': 'Zebra', 'Name': 'Ricky', 'Weight': 100}]
    both_eaters = [{'Animal': 'Wolf', 'Name': 'Sky', 'Weight': 78},
                   {'Animal': 'Piranha', 'Name': 'Lily', 'Weight': 0.5}]
    expected_total_cost = (
        160 * 0.10 * 12.56 +  # Lion
        200 * 0.08 * 5.60 +   # Giraffe
        150 * 0.09 * 12.56 +  # Tiger
        100 * 0.08 * 5.60 +   # Zebra
        78 * 0.07 * (12.56 * 0.9 + 5.60 * 0.1) +  # Wolf
        0.5 * 0.5 * (12.56 * 0.5 + 5.60 * 0.5)   # Piranha
    )
    test_feeding_cost(meat_eaters, fruit_eaters, both_eaters, expected_total_cost)

test_all_types_of_eaters()

Expected: 571.96744 Actual: 571.96744


Проверка вручную и по заданной функции задали **верный** результат


---



3.5 Проверим выдачу ошибок

Введем новые параметры

Проверка получения ошибки при отрицательном значении веса в zoo.csv

In [25]:
def test_negative_weight():
    zoo_data = pd.DataFrame([{'Animal': 'Lion', 'Name': 'Leo', 'Weight': 160},
                             {'Animal': 'Tiger', 'Name': 'Max', 'Weight': -150}])
    animals_data = pd.DataFrame({
        'Animal': ['Lion', 'Tiger'],
        'Weight_coef': [0.10, 0.09],
        'Food': ['meat', 'meat'],
        'Proportion': [np.nan, np.nan]
    })
    prices_data = pd.DataFrame({
        'Food': ['meat', 'fruit', 'both'],
        'Price': [12.56, 5.60, np.nan]
    })

    with pytest.raises(ValueError, match="incorrect data in data source"):
        calculate_feeding_cost(zoo_data, animals_data, prices_data)

test_negative_weight()

Failed: DID NOT RAISE <class 'ValueError'>

Проверка получения ошибки при нулевом значении веса в zoo.csv


In [26]:
def test_zero_weight():
    zoo_data = pd.DataFrame([{'Animal': 'Lion', 'Name': 'Leo', 'Weight': 160},
                             {'Animal': 'Tiger', 'Name': 'Max', 'Weight': 0}])
    animals_data = pd.DataFrame({
        'Animal': ['Lion', 'Tiger'],
        'Weight_coef': [0.10, 0.09],
        'Food': ['meat', 'meat'],
        'Proportion': [np.nan, np.nan]
    })
    prices_data = pd.DataFrame({
        'Food': ['meat', 'fruit', 'both'],
        'Price': [12.56, 5.60, np.nan]
    })

    with pytest.raises(ValueError, match="incorrect data in data source"):
        calculate_feeding_cost(zoo_data, animals_data, prices_data)

test_zero_weight()

Failed: DID NOT RAISE <class 'ValueError'>

Проверка получения ошибки при некорректном синтаксисе данных в zoo.csv

In [27]:
def test_incorrect_syntax():
    zoo_data = pd.DataFrame([{'Animal': 'Lion', 'Name': 'Leo', 'Weight': 160},
                             {'Animal': 'Tiger', 'Name': 'Max', 'Weight': 'Incorrect syntax'}])
    animals_data = pd.DataFrame({
        'Animal': ['Lion', 'Tiger'],
        'Weight_coef': [0.10, 0.09],
        'Food': ['meat', 'meat'],
        'Proportion': [np.nan, 'invalid']
    })
    prices_data = pd.DataFrame({
        'Food': ['meat', 'fruit', 'both'],
        'Price': [12.56, 5.60, np.nan]
    })

    with pytest.raises(ValueError, match="incorrect syntax in data source"):
        calculate_feeding_cost(zoo_data, animals_data, prices_data)

test_incorrect_syntax()

TypeError: can't multiply sequence by non-int of type 'float'